<a href="https://colab.research.google.com/github/st24hour/tutorial/blob/master/dcgan_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deep Convolutional Generative Adversarial Network

이 튜토리얼에서 우리는 [Deep Convolutional Generative Adversarial Network](https://arxiv.org/pdf/1511.06434.pdf) (DCGAN)을 사용하여 어떻게 handwritten 숫자들을 만드는 네트워크를 학습시킬지를 배울 것입니다.


## GAN이란?
[Generative Adversarial Networks](https://arxiv.org/abs/1406.2661) (GANs) 는 최근 computer science 분야에서 가장 흥미로운 아이디어 중에 하나입니다. 두 개의 모델은 적대적인 과정을 통해 동시다발적으로 훈련됩니다. *Generator* ("예술가")는 진짜처럼 보이는 이미지를 만드는 법을 배우고, *Discriminator*("비평가")는 진짜 이미지와 가짜 이미지를 구분하는 법을 배웁니다.

![A diagram of a generator and discriminator](https://tensorflow.org/beta/tutorials/generative/images/gan1.png)

*Generator*는 점진적으로 더 진짜 같은 이미지를 만들게 되고 *Discriminator*는 더 잘 진짜와 가짜를 구분하게 됩니다. 

![A second diagram of a generator and discriminator](https://tensorflow.org/beta/tutorials/generative/images/gan2.png)

이 코드를 통해 우리는 MNIST 데이터셋을 사용하여 GAN을 학습해볼 것입니다. 아래의 애니메이션은 *Generator*로부터 50 epoch을 거쳐 만들어진 여러 개의 이미지들입니다. 이 이미지들은 random noise로부터 시작해서 점점 시간이 갈 수록 handwritten 숫자의 형상을 갖추게 됩니다.

![sample output](https://tensorflow.org/images/gan/dcgan.gif)



## DCGAN 튜토리얼

이제부터 어떻게 네트워크를 만들고 GAN학습을 시킬지를 배워봅시다.
순서는 다음과 같습니다.
1. Tensorflow와 다른 library들을 불러온다.
2. 데이터셋을 불러온다.
3. 모델을 만든다.
4. loss와 optimizer를 정의한다.
5. Training loop를 정의한다.
6. Training!
7. Test

그럼 하나씩 진행해보도록 합시다.



### Import Tensorflow and other libraries




In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
!pip install tensorflow-gpu==2.0.0-beta1 #tensorflow gpu 버전을 설치합니다


In [0]:
import tensorflow as tf # tensorflow를 import해줍니다

In [0]:
tf.__version__ # 내가 사용할 tensorflow의 버전을 나타냅니다

In [0]:
# 나중에 GIF를 생성하기 위해서 import해줍니다
!pip install imageio

In [0]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display

### Load and prepare the dataset

MNIST 데이터셋을 이용하여 Generator와 Discriminator를 학습시켜봅시다. 학습이 끝나면 Generator는 MNIST 데이터를 닮은 handwritten 숫자들을 생성할 것입니다.

주어진 정보들을 이용하여 빈 칸을 채워보세요! ([    ]가 빈 칸을 나타냅니다. 괄호를 지우고 알맞은 코드를 써주세요)

1. MNIST 데이터는 28x28x1의 dimension을 갖고 있습니다. 받아온 train_images를 (train_image의 갯수, width, height, channel)로 reshape한 뒤 float32로 casting해줍니다.
2. [0, 255] 범위로 이루어진 train_images를 [-1, 1]의 범위로 normalize해주기 위해 이미지에서 127.5를 뺀 뒤 127.5로 나눠줍니다. train_images에서 바로 빼고 나눠주시면 됩니다.
(픽셀의 값이 0일 경우 (0 - 127.5)/127.5=-1, 255일 경우 (255 - 127.5)/127.5=1이 됩니다. 이 과정을 통해  기존의 0과 255사이의 값으로 이루어진 이미지가 -1과 1사이의 값을 가지도록 normalize를 해주는 것입니다.)
3. BUFFER SIZE는 60000입니다.
4. BATCH SIZE는 256입니다.

In [0]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [0]:
train_images = train_images.reshape([      ]).astype('[      ]') # 1번
train_images = [             ] # 2번

In [0]:
BUFFER_SIZE = [          ] # 3번
BATCH_SIZE = [          ] # 4번

In [0]:
# 데이터를 shuffle하고 batch를 받아옵니다
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Create the models
이제 Generator와 Discriminator 모델을 만들어봅시다!
Generator와 Discriminator들은 모두  [Keras Sequential API](https://www.tensorflow.org/guide/keras#sequential_model).를 사용하여 정의할 것입니다.

우리가 만들 Generator와 Discriminator의 구조는 아래 그림들과 같습니다.

빈 칸을 채워서 그림과 맞는 모델을 만들어보세요.

### The Generator

Generator는 `tf.keras.layers.Conv2DTranspose` (unsampling) 레이어를 사용하여 seed(random noise)로부터 이미지를 생성합니다. 먼저 `Dense` 레이어에서 이 seed를 input으로 받은 뒤에 원본 이미지 크기(28x28x1)에 도달할 때까지 unsampling을 여러번 반복합니다. `tf.keras.layers.BatchNormalization`과 `tf.keras.layers.LeakyReLU` activation을 각 블럭의 마지막에 추가해줍니다.
![generator](https://docs.google.com/uc?export=download&id=12hZMMHiHbqsiDwGKi7J87T-Ic_y8s6p5)

In [0]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense([      ], use_bias=False, input_shape=([     ],))) # 빈칸을 채워주세요
    model.add(layers.[       ]()) # 빈칸을 채워주세요
    model.add(layers.[       ]()) # 빈칸을 채워주세요

    model.add(layers.Reshape(([      ]))) # 빈칸을 채워주세요
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose([    ], ([     ]), strides=([     ]), padding='same', use_bias=False)) # 빈칸을 채워주세요
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.[       ]()) # 빈칸을 채워주세요
    model.add(layers.[       ]()) # 빈칸을 채워주세요

    model.add(layers.Conv2DTranspose([    ], ([     ]), strides=([     ]), padding='same', use_bias=False)) # 빈칸을 채워주세요
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.[       ]() # 빈칸을 채워주세요
    model.add(layers.[       ]())# 빈칸을 채워주세요

    model.add(layers.Conv2DTranspose([     ], ([     ]), strides=([      ]), padding='same', use_bias=False, activation='tanh')) # 빈칸을 채워주세요
    assert model.output_shape == (None, 28, 28, 1)

    return model

Generator를 사용하여 이미지를 생성하도록 합니다.
1. 1x100 dimension을 가진 noise를 정의해줍시다. 
2. 만들어진 이미지 generated_image는 generator에 noise를 input으로 넣어줌으로써 생성됩니다.

In [0]:
generator = make_generator_model()

noise = tf.random.normal([     ]) # 1번
generated_image = generator([     ], training=False) # 2번

plt.imshow(generated_image[0, :, :, 0], cmap='gray')

### The Discriminator
Discriminator는 CNN-based 이미지 분류기입니다.

빈 칸을 채워 Discriminator의 구성을 완료합시다.
![generator](https://docs.google.com/uc?export=download&id=18fiJbjonI34sO0CJJ-XxhLHXpFBIAlnU)

In [0]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D([     ], ([     ]), strides=([      ]), padding='same',
                                     input_shape=[28, 28, 1])) # 빈칸을 채워주세요
    model.add(layers.[       ]()) # 빈칸을 채워주세요
    model.add(layers.[       ]([   ])) # 빈칸을 채워주세요

    model.add(layers.Conv2D([    ], ([    ]), strides=([      ]), padding='same')) # 빈칸을 채워주세요
    model.add(layers.[       ]()) # 빈칸을 채워주세요
    model.add(layers.[       ]([   ])) # 빈칸을 채워주세요

    model.add(layers.[     ]()) # 빈칸을 채워주세요
    model.add(layers.[     ](1)) # 빈칸을 채워주세요

    return model

Discriminator(아직 training하지 않은)를 사용하여 만들어진 이미지가 진짜인지 가짜인지 구별하도록 합니다. 모델은 진짜 이미지를 받았을 때는 positivie한 값을 가짜 이미지를 받았을 때는 negative한 값을 내보내도록 학습될 것 입니다.

1. Discriminator(discriminator)가 만들어진 이미지(generated_image)를 받아 결정값을(decision) 내보내도록 합시다.

In [0]:
discriminator = make_discriminator_model()
decision = discriminator([      ]) # 1번
print (decision)

## Define the loss and optimizers

두 모델을 위해 loss functions과 optimizer를 정의해줍시다.

In [0]:
# 이 메서드는 크로스 엔트로피 손실함수 (cross entropy loss)를 계산하기 위해 헬퍼 (helper) 함수를 반환합니다.
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

### Discriminator loss

이 loss는 Discriminator가 얼마나 잘 진짜 이미지와 가짜 이미지를 구별해내는 지를 보여줄 것입니다. Discriminator가 진짜 이미지를 받아서 예측해낸 값과 1로 이루어진 array를 비교하고, 가짜 이미지(만들어진 이미지)를 받아서 예측해낸 값과 0으로 이루어진 array를 비교할 것입니다.

1. real_output과 같은 크기를 가진 1로 이루어진 array와 real_output과의 cross entropy를 계산하여 real_loss라고 정의해줍시다. (Hint: tf.ones_like)
2. fake_output과 같은 크기를 가진 0로 이루어진 array와 fake_output과의 cross entropy를 계산하여 fake_loss라고 정의해줍시다. (Hint: tf.zeros_like)
3. real_loss와 fake_loss를 합해 total_loss라고 정의해줍시다.

In [0]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy([       ]) # 1번
    fake_loss = cross_entropy([       ]) # 2번
    total_loss = [       ] # 3번
    return total_loss

### Generator loss

Generator의 loss는 얼마나 discriminator를 잘 속였는지를 값으로 보여줍니다. 직관적으로, 만약 Generator가 잘 하고 있다면 discriminator는 가짜 이미지를 진짜라고 판단할 것입니다 (or 1). 여기서 우리는 Generator가 만든 이미지(fake_output)이 Discriminator에 들어가서 나온 값이 1에 가까워지도록 할 것입니다.
1. fake_output과 같은 크기를 가진 1로 이루어진 array와 fake_output과의 cross entropy를 계산하는 함수를 정의해줍시다.

In [0]:
def generator_loss(fake_output):
    return cross_entropy([      ], fake_output) # 1번

Discriminator와 Generator는 각각 학습되기 때문에 다른 optimizer를 정의해주어야합니다.
1. 두 개의 optimizer를 각각 Adam Optimizer를 learning rate 1e-4로 정의해줍시다. ( Hint: tf.keras.optimizers.Adam(*learning_rate*))

In [0]:
generator_optimizer = [      ] # 1번
discriminator_optimizer = [      ] # 1번

### Save checkpoints

모델을 저장하고 다시 불러오기 위해 checkpoint를 만들어줍시다.

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

## Define the training loop

1. 우리는 50 epoch을 돌려 training 시킬 것입니다.
2. 우리가 사용할 noise의 dimension은 100입니다.

In [0]:
EPOCHS = [    ] # 1번
noise_dim = [    ] # 2번
num_examples_to_generate = 16


# animated GIF를 생성하기 위한 seed 생성
seed = tf.random.normal([num_examples_to_generate, noise_dim])

Training loop는 Generator가 random noise를 input으로 받으면서부터 시작됩니다. seed는 이미지를 만들기 위해 쓰입니다. Discriminator는 진짜 이미지(Training set에 있는)와 가짜 이미지(Generator가 만든)를 구분하도록 학습합니다. 각 model의 loss를 각각 계산한뒤 gradient를 이용하여 Generator와 Discriminator를 업데이트합시다.

1. Batch Size x Noise Dimension 크기의 noise를 생성합시다. (Hint: tf.random.normal)
2. 위에서 정의한 generator에 noise를 input으로 주어 generated_images를 생성합니다. (Hint: training=True를 설정해줍시다)
3. 위에서 정의한 discriminator에 진짜 이미지(images)를 넣어 real_output을 만듭니다. (Hint: training=True를 설정해줍시다)
4. 위에서 정의한 discriminator에 가짜 이미지를 넣어(generated_images) fake_output을 만듭니다. (Hint: training=True를 설정해줍시다)
5. 위에서 정의한 generator_loss에 fake_output을 넣어 gen_loss를 계산합니다.
5. real_output과 fake_output을 이용하여 disc_loss를 계산합니다.


In [0]:
# `tf.function`이 어떻게 사용되는지 주목해 주세요.
# 이 데코레이터는 함수를 "컴파일"합니다.
@tf.function
def train_step(images):
    noise = [      ] # 1번

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = [      ] # 2번

      real_output = [      ] # 3번
      fake_output = [      ] # 4번

      gen_loss = [      ] # 5번
      disc_loss = [      ] # 6번

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables)) 

## Train 함수를 정의해봅시다

In [0]:
def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()

    for image_batch in dataset:
      train_step(image_batch)

    # Produce images for the GIF as we go
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                             epoch + 1,
                             seed)

    # Save the model every 15 epochs
    if (epoch + 1) % 15 == 0:
      checkpoint.save(file_prefix = checkpoint_prefix)

    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

  # Generate after the final epoch
  display.clear_output(wait=True)
  generate_and_save_images(generator,
                           epochs,
                           seed)

**Generate and save images**



In [0]:
def generate_and_save_images(model, epoch, test_input):
  # Notice `training` is set to False.
  # This is so all layers run in inference mode (batchnorm).
  predictions = model(test_input, training=False)

  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

## Train the model

위에서 정의한 `train()` 함수를 불러 Generator와 Discriminator를 동시에 학습합니다. GAN을 학습하는 것은 까다롭습니다. Generator와 Discriminator가 서로를 overpower하지 않도록 해주는 것이 중요합니다.

Training을 시작할 때 만들어진 이미지는 random한 noise처럼 보입니다. 그러나 학습이 진행되면서 만들어진 숫자들은 진짜처럼 보이기 시작합니다. 50 epoch이 지나면 MNIST 숫자를 닮은 이미지가 생성될 것 입니다.

1. 위에서 정의한 train 함수에 넣어줘야하는 인자를 넣어 학습을 시작해줍시다.

In [0]:
%%time
train([    ], [    ]) # 1번

학습이 다 끝나면 가장 최근의 checkpoint를 불러 test해봅시다.

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## Create a GIF


In [0]:
# Display a single image using the epoch number
def display_image(epoch_no):
  return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [0]:
display_image(EPOCHS)

`imageio` 라이브러리를 사용하여 training 도중 저장 된 이미지들로 gif를 만들어봅시다.


In [0]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('image*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    frame = 2*(i**0.5)
    if round(frame) > round(last):
      last = frame
    else:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)

In [0]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download(anim_file)

## Report


1. 레포트에 학습시킨 모델의 1epoch, 5epoch, 15epoch, 30epoch, 50epoch의 결과 이미지들을 첨부해주세요.
2. noise의 dimension을 10, 50, 200으로 바꾸어 학습시켜본 뒤 결과 이미지를 첨부해주세요.

juseung_yun@kaist.ac.kr로 레포트를 제출해주세요!